# Python Text Analysis: Topic Modeling

We've considered so far how to create numerical representations of words. We've even performed sentiment classification with those numerical representations. How else can we leverage these numerical representations to elucidate structure from natural language?

In this session, we're going to discuss *topic modeling*. In topic modeling, we aim to discover how the documents in a corpus may be modeled as a function of specific topics. This is not the same thing as direct clustering, though, in which we might directly assign each document to a particular cluster.

Consider genre classification. Some books may neatly fall into one genre, such as mystery, science fiction, etc. However, other books may be considered as incorporating multiple genres. You might have a fantasy novel which has mystery components to it, or a romance novel set in the future. In these cases, we don't want to cluster the fantasy novel into a "fantasy" bucket, and the romance novel in a "romance" bucket. We'd instead like to have some measure of assigning various topics, with different magnitudes to documents. This is the goal of topic modeling.

We will use two approaches to perform topic modeling on the same corpus: non-negative matrix factorization, and latent dirichlet allocation.

# Dataset: 20 Newsgroups

We will be using a new dataset called the **20 Newsgroups** dataset. You can find the original page for this dataset [here](http://qwone.com/~jason/20Newsgroups/).

This dataset is comprised of around 18000 newsgroups posts on 20 topics. The split between the train and test set is based upon a messages posted before and after a specific date. The news groups are as follows, with specific labels indicated:

* *Computers*
    * comp.graphics
    * comp.os.ms-windows.misc
    * comp.sys.ibm.pc.hardware
    * comp.sys.mac.hardware
    * comp.windows.x
* *Recreation*
    * rec.autos
    * rec.motorcycles
    * rec.sport.baseball
    * rec.sport.hockey
* *Science*
    * sci.crypt
    * sci.electronics
    * sci.med
    * sci.space
* *Miscellaneous*
    * misc.forsale
* *Politics*
    * talk.politics.misc
    * talk.politics.guns
    * talk.politics.mideast
* *Religion*
    * talk.religion.misc
    * alt.atheism
    * soc.religion.christian
    
Let's begin by importing the dataset. We'll use `scikit-learn` to do so.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

%matplotlib inline

In [ ]:
# Import fetcher function
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# Always check the documentation!
data, labels = fetch_20newsgroups(
    subset='all',
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True)

Let's take a look at some of the data samples:

In [ ]:
print(data[1000])
print('--------')
print(data[2000])
print('--------')
print(data[3000])

If we take a look at the labels, we see that they're integers, each specifying one of the 20 possible classes:

In [ ]:
print(np.unique(labels))
print(labels.shape)

We can access the corresponding names of these labels by using a different keyword argument in the original `fetch_20newsgroups` call:

In [ ]:
newsgroups = fetch_20newsgroups(
    subset='all',
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"))

In [ ]:
list(newsgroups)

In [ ]:
newsgroups.target_names

# Non-negative Matrix Factorization

Non-negative matrix factorization (NMF) is a dimensionality reduction technique that can be used to perform topic modeling. What does it mean? Non-negative implies we're not working with any negative numbers, and matrix factorization implies we're taking a matrix, and breaking it down into "factors".

The matrix under consideration is going to be some numerical representation of the text. We've already considered one - the document term matrix! Specifically, we're going to build a TF-IDF matrix from the data - let's call this matrix $V$. Let's remind ourselves what $V$ is: it's a $D\times T$ matrix, where $D$ is the number of documents, and $T$ is the number or terms, or tokens. 

![NMF](../images/nmf.png)

The goal in NMF is to write this matrix as a product of two matrices,

$$
V \approx WH
$$

where $W$ is a $D\times K$ matrix and $H$ is a $K\times T$ matrix. So, in matrix $W$, we can still consider rows as corresponding to documents, and in $H$, we can think of columns as corresponding to terms. But what about $K$, the inner dimension?

We can think of $K$ as enumerating *topics*. If $K$ corresponds to topics, then each row of $H$ corresponds to a different topic. We can interpret $H$ as enumerating what contribution each *term* makes to each *topic*. For example, if the first row of the NMF only has non-zero entries for terms `soccer`, `basketball`, and `baseball`, we might reasonably conclude that the topic corresponds to "sports". The numbers for each entry indicate the contribution of that term to the topic - so if the topic is mainly baseball, it might have a higher value in that entry.

What does this mean for $W$? It's detailing how each *document* (rows) break down into *topics* (columns). So, we can think of NMF as estimating the contribution different topics to a specific document. In newsgroups, we might expect there to be a large contribution from that "sports" topic above into the samples labeled as "rec.sports.baseball".

The breakdown of the original matrix into $W$ and $H$ can only be interpreted if there are no negative entries in any matrix. We already know a TF-IDF DTM is going to be non-negative. If we guarantee that $W$ and $H$ are as well, we can quite literally think of this factorization is creating building blocks for each document!

There's a small issue here: we can get the breakdown of documents into topics, but it's up to us to "interpret" what the topics might be. This can be tricky business, as we might see. Furthermore, we don't even know how many topics we should pick! There are procedures to identify a good number of topics, but at some level it is subjective.

Let's try fitting an NMF to the newsgroups data. First, we need to use `TfidfVectorizer` to transform the data into a document term matrix (remember how to do this?).

In [ ]:
# Is this a good scenario to be removing stop words?
n_tokens = 1000

vectorizer = TfidfVectorizer(
    max_df=0.95,
    min_df=2,
    max_features=n_tokens,
    stop_words="english")

In [ ]:
# Perform vectorizing
tfidf = vectorizer.fit_transform(data)
tokens = vectorizer.get_feature_names_out()

In [ ]:
# How many samples, and how many tokens?
tfidf.todense().shape

Let's look at the tokens with highest TF-IDF scores:

In [ ]:
tfidf_df = pd.DataFrame(
    tfidf.todense(),
    columns=tokens)
tfidf_df.sum(axis=0).sort_values(ascending=False).head(30)

We can perform NMF using the `NMF` module from `scikit-learn`:

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
n_components = 10
random_state = 1

nmf = NMF(
    n_components=n_components,
    random_state=random_state,
    alpha=0.1,
    l1_ratio=0.5,
    init='nndsvda',
    max_iter=500).fit(tfidf)

We can take a look at the $H$ matrix, or the topics, by examining the `components_` member variable:

In [ ]:
print(nmf.components_.shape)

Let's take a peek at the distribution of values in the first topic:

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.bar(x=np.arange(n_tokens), height=nmf.components_[0])

This is nice, but it'd be nice to look at the top tokens - the large spikes we see there - and ignore all the smaller contributions to a topic. We'll use a plotter function which will nicely show the largest contributions to each topic, as well as what the corresponding tokens are:

In [ ]:
def plot_top_words(model, feature_names, n_top_words=10, n_row=2, n_col=5):
    fig, axes = plt.subplots(n_row, n_col, figsize=(3 * n_col, 5 * n_row), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 20})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)

        for i in "top right left".split():
            ax.spines[i].set_visible(False)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)

    return fig, axes

In [ ]:
fig, axes = plot_top_words(nmf, tokens)
plt.show()

What can we tell from this? For one, Topic 3 looks very much like a topic related to religion; Topic 4 looks like it's about sports; Topic 7 looks like it might be related to computers; and Topic 10 looks realted to politics (specifically, Israeli or generally Middle Eastern politics).

Some of the topics look a little bit harder to interpret, and that comes with the territory in topic modeling.

We were able to get the $H$ matrix, but what about the $W$ matrix? To get this, we need to *transform* the DTM into the *basis* created by the NMF factorization. We can do this with the `transform` function:

In [ ]:
W = nmf.transform(tfidf)
print(W.shape)

Let's take a look at a random entry:

In [ ]:
entry = 999
print(data[999])

The accompanying label for this entry can be obtained from the `newsgroups` variable we created earlier:

In [ ]:
newsgroups.target_names[labels[999]]

Let's examine the breakdown of this entry by topic:

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.bar(x=np.arange(n_components), height=W[999])
ax.set_xticks(np.arange(n_components))
ax.set_xticklabels(np.arange(n_components) + 1)
ax.set_xlabel('Topic', fontsize=15)

The biggest contribution is topic 7, which makes some sense: it generally seems to deal with computers. Notice that many of the topics have been zeroed out entirely - this is a product of enforcing *sparsity* in the model, and it stems from the `alpha` parameter in the NMF.

## Challenge 1: Exploring Hyperparameters in NMF

The choice of 10 components was somewhat arbitrary. It was something we had to do before we could proceed with fitting the model to the data. This is what's known as a *hyperparameter*. There are other hyperparameters in the `NMF`. For example, the `alpha` values specifies to what degree we should force values to be set equal to zero.

Try fitting the NMF with other variations of hyperparameters, and plot the resulting topics using the `plot_top_words` function. What do you notice?

# Latent Dirichlet Allocation

![lda](../images/lda.png)